### Workflow 

1. Use the labeled Dataset class that you wrote for milestone 2 and separate the images in the labeled folder into **train** and **validation** sets (having a validation set is especially important when training on a limited number of samples, since the danger of overfitting to training data is high).
1. Construct and train your supervised melanoma classifier. How does it perform on the test set?
1. Transfer learning is a popular machine learning (ML) approach when training data is scarce. **Either choose a pre-trained model out of the ones available in the `torchvision.models` package, or train your own on a (large) dataset of your choice, and fine-tune it on your melanoma data.** How does its performance compare to the fully supervised classifier that you trained from scratch in step 2?
1. When transfer learning does not prove to be sufficient, another option that we have is to take advantage of the unlabeled data by employing a semi-supervised ML strategy. Semi-supervised learning is a rapidly developing field, so the best place to look for the state-of-the-art methods is the famous arXiv.org, an open-access archive.  containing thousands of articles across numerous scientific disciplines, including computer science. Learning to work with the arXiv is an important skill for ML engineers involved in solving non trivial problems. **Spend some time to find at least 2-3 different approaches that have shown to be successful in semi-supervised ML for computer vision applications, and list them** along with the arXiv references that you found. It's fine to search arxiv.org and just read abstracts. See
  * https://arxiv.org/search/?query=semi+supervised+image+classification&searchtype=all



### Resources

* https://livebook.manning.com/book/grokking-deep-learning-for-computer-vision/chapter-6
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* https://livebook.manning.com/book/deep-learning-with-pytorch/chapter-2/81
* [Learning Rate Schedulers - Pytorch](https://www.kaggle.com/isbhargav/guide-to-pytorch-learning-rate-scheduling)
* [Would Transfer Learning on ImageNet work?](https://pytorch.org/serve/model_zoo.html)

##### Convenience

* https://www.youtube.com/watch?v=RKHopFfbPao (try tpdm)

In [61]:
url = 'https://liveproject-resources.s3.amazonaws.com/other/MelanomaDetection.zip'
pth = './data/MelanomaDetection.zip'
unp = './data/MelanomaDetection'
bs  = 24

In [62]:
import torch
import torchvision
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss, BCEWithLogitsLoss, CrossEntropyLoss #not sure which I'd like to use. 
from torch.nn import Linear
from torch.optim import Adam, SGD
from torch.utils.data import Dataset, DataLoader, Subset, SubsetRandomSampler
from torch.optim.lr_scheduler import CyclicLR, OneCycleLR

import numpy as np

In [63]:
%load_ext autoreload
%autoreload 2

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
#### Ensure we are running on the GPU

# If it's not running you may need to reboot. 

In [65]:
torch.cuda.is_available()

True

In [27]:
gpu_device = torch.device("cuda:0")

### Create Training and Validation Sets

In [28]:
labeledSet = MILabeled(unp + '/labeled', ToTensor())

sd = 43
cs = len(labeledSet)
ci = range(0, cs)
train_idxs, test_idxs = train_test_split(ci, random_state=sd, train_size=0.75)

train_sampler = SubsetRandomSampler(train_idxs) #subset would work just fine here too I think. 
test_sampler  = SubsetRandomSampler(test_idxs)


In [29]:
labeled_ds = MILabeled(unp + '/labeled', ToTensor())
train_dl = DataLoader(labeled_ds, batch_size=bs, sampler=train_sampler)
test_dl = DataLoader(labeled_ds, batch_size=bs, sampler=test_sampler)

### Construct the Model

In [53]:
mod = torchvision.models.resnet18(False)


In [54]:
print(mod)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [55]:
mod.fc

Linear(in_features=512, out_features=1000, bias=True)

In [56]:
#swap out the fully connected layer.  
mod.fc = Linear(512, 2)

In [57]:
mod.to(gpu_device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [58]:
num_epochs=3000

criterion = CrossEntropyLoss() # LogSoftmax + NLLLoss
optimizer = Adam(mod.parameters(), lr=0.1)
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_dl), epochs=num_epochs)

### Write a Training Loop

In [59]:

def train_epoch(epoch, model, dataloader, criterion, optimizer, scheduler):
    
    #set train to true
    model.train()
    
    loss_items = []
    acc_list   = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):

        labels = labels.to(gpu_device, torch.long)
        images = images.to(gpu_device, torch.float)
        
        model.zero_grad()

        pred = model(images)
       
        #https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/2 (Notes that this is why you use CrossEntropyLoss fn)
        loss = criterion(pred, labels)
        
        loss_items.append(loss.item())
        
        plab = torch.argmax(pred, axis=1) 
        acc  = plab.eq(labels).sum().item() / bs
        
        acc_list.append(acc)

        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        #print batch results here
        
    #print epoch results here
    print('TRAIN | Epoch Loss: {}'.format(np.mean(loss_items)))
    print('TRAIN | Epoch Accuracy: {}'.format(np.mean(acc_list)))




def valid_epoch(epoch, model, dataloader, criterion, bs):
    
    #set train to false
    model.eval()
    
    loss_items = []
    acc_list   = []
    
    #loop through batches in data loader
    for bi, (labels, images) in enumerate(dataloader):
    
        labels = labels.to(gpu_device, torch.long)
        images = images.to(gpu_device, torch.float)
    
        with torch.no_grad():
            pred = model(images)
    
    
        loss = criterion(pred, labels)
        loss_items.append(loss.item())
        
        #calculate the accuracy
        #https://discuss.pytorch.org/t/torchvision-models-dont-have-softmax-layer/18071/4
        #Confirmed that you can argmax on the raw outputs of a dense layer. No need to Softmax them. 
        plab = torch.argmax(pred, axis=1) 
        acc  = plab.eq(labels).sum().item() / bs
        
        assert acc >= 0 and acc <= 1.0, "Your accuracy should be between 0 and 1."
        
        #print batch results here
        acc_list.append(acc)
    
    #print epoch results here
    print('VALID | Epoch Loss: {}'.format(np.mean(loss_items)))
    print('VALID | Epoch Accuracy: {}'.format(np.mean(acc_list)))
 




In [60]:

for i in range(num_epochs):
    
    
    train_epoch(i, mod, train_dl, criterion, optimizer, scheduler)
    valid_epoch(i,  mod, test_dl,  criterion, bs)

    


TRAIN | Epoch Loss: 0.5834162150110517
TRAIN | Epoch Accuracy: 0.6369047619047619
VALID | Epoch Loss: 1.8202130397160847
VALID | Epoch Accuracy: 0.3888888888888889
TRAIN | Epoch Loss: 0.5699373739106315
TRAIN | Epoch Accuracy: 0.755952380952381
VALID | Epoch Loss: 2.2012688318888345
VALID | Epoch Accuracy: 0.3888888888888889
TRAIN | Epoch Loss: 0.231510015470641
TRAIN | Epoch Accuracy: 0.7976190476190477
VALID | Epoch Loss: 5.023758729298909
VALID | Epoch Accuracy: 0.3888888888888889
TRAIN | Epoch Loss: 0.3567576419029917
TRAIN | Epoch Accuracy: 0.7797619047619049
VALID | Epoch Loss: 0.9192746877670288
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.23179908895066806
TRAIN | Epoch Accuracy: 0.8333333333333333
VALID | Epoch Loss: 1.0439581473668416
VALID | Epoch Accuracy: 0.5138888888888888
TRAIN | Epoch Loss: 0.17601212859153748
TRAIN | Epoch Accuracy: 0.8154761904761906
VALID | Epoch Loss: 0.9728821714719137
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Los

TRAIN | Epoch Loss: 0.007199134162094977
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 0.9485640525817871
VALID | Epoch Accuracy: 0.5416666666666666
TRAIN | Epoch Loss: 0.5865515680135494
TRAIN | Epoch Accuracy: 0.875
VALID | Epoch Loss: 0.9512859781583151
VALID | Epoch Accuracy: 0.5416666666666666
TRAIN | Epoch Loss: 0.062473068229987154
TRAIN | Epoch Accuracy: 0.869047619047619
VALID | Epoch Loss: 1.7575095494588215
VALID | Epoch Accuracy: 0.4444444444444444
TRAIN | Epoch Loss: 0.09730551418449197
TRAIN | Epoch Accuracy: 0.863095238095238
VALID | Epoch Loss: 0.9299316803614298
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.08654052012466959
TRAIN | Epoch Accuracy: 0.8630952380952381
VALID | Epoch Loss: 0.9968603452046713
VALID | Epoch Accuracy: 0.49999999999999994
TRAIN | Epoch Loss: 0.09958004525729588
TRAIN | Epoch Accuracy: 0.869047619047619
VALID | Epoch Loss: 2.1655731002489724
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.27

TRAIN | Epoch Loss: 0.22110112330743245
TRAIN | Epoch Accuracy: 0.8511904761904763
VALID | Epoch Loss: 1.5245825449625652
VALID | Epoch Accuracy: 0.37500000000000006
TRAIN | Epoch Loss: 0.1092277251716171
TRAIN | Epoch Accuracy: 0.8571428571428571
VALID | Epoch Loss: 0.557561198870341
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.08780906881604876
TRAIN | Epoch Accuracy: 0.875
VALID | Epoch Loss: 0.6095931927363077
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.07553707914693016
TRAIN | Epoch Accuracy: 0.8630952380952381
VALID | Epoch Loss: 0.6981073419253031
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.03369833222989525
TRAIN | Epoch Accuracy: 0.886904761904762
VALID | Epoch Loss: 0.7934528390566508
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.022865107881703546
TRAIN | Epoch Accuracy: 0.8809523809523808
VALID | Epoch Loss: 1.651500642299652
VALID | Epoch Accuracy: 0.5000000000000001
TRAIN | Epoch Loss: 0.04076

TRAIN | Epoch Loss: 0.0950244175536292
TRAIN | Epoch Accuracy: 0.8630952380952381
VALID | Epoch Loss: 0.5900669495264689
VALID | Epoch Accuracy: 0.5138888888888888
TRAIN | Epoch Loss: 0.12348839587398938
TRAIN | Epoch Accuracy: 0.8273809523809524
VALID | Epoch Loss: 0.6285788615544637
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.05299503542482853
TRAIN | Epoch Accuracy: 0.875
VALID | Epoch Loss: 0.7842290103435516
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.2400455486827663
TRAIN | Epoch Accuracy: 0.869047619047619
VALID | Epoch Loss: 0.848378856976827
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.038697199895977974
TRAIN | Epoch Accuracy: 0.8869047619047619
VALID | Epoch Loss: 2.315827568372091
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.16688496140497072
TRAIN | Epoch Accuracy: 0.869047619047619
VALID | Epoch Loss: 1.01581609249115
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 0.0547215761

TRAIN | Epoch Loss: 0.030573519761674106
TRAIN | Epoch Accuracy: 0.8809523809523808
VALID | Epoch Loss: 2.0646571119626365
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.07519839207608518
TRAIN | Epoch Accuracy: 0.8869047619047619
VALID | Epoch Loss: 1.355135202407837
VALID | Epoch Accuracy: 0.5138888888888888
TRAIN | Epoch Loss: 0.11375724887641679
TRAIN | Epoch Accuracy: 0.8571428571428571
VALID | Epoch Loss: 1.660578469435374
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.01736704183609358
TRAIN | Epoch Accuracy: 0.886904761904762
VALID | Epoch Loss: 2.6234057545661926
VALID | Epoch Accuracy: 0.5
TRAIN | Epoch Loss: 0.01049662847071886
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.0833401282628377
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 0.3426036924522902
TRAIN | Epoch Accuracy: 0.880952380952381
VALID | Epoch Loss: 2.108069916566213
VALID | Epoch Accuracy: 0.5277777777777778
TRAIN | Epoch Loss: 0.20768284

...

TRAIN | Epoch Loss: 7.77074248097571e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.998150944709778
VALID | Epoch Accuracy: 0.5000000000000001
TRAIN | Epoch Loss: 5.8547371736494824e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.663342038790385
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 5.539116422629117e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.70101797580719
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.00010548177202248812
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.9984357754389444
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.0001368763878417667
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.6445270379384358
VALID | Epoch Accuracy: 0.513888888888889
TRAIN | Epoch Loss: 6.485553006184221e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.6361380219459534
VALID | Epoch Accuracy: 0.4861111111111111

TRAIN | Epoch Loss: 4.0136632898273615e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.2617234786351523
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 2.9470238977020407e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 5.234803557395935
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 3.4161976145696826e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 3.3096685806910195
VALID | Epoch Accuracy: 0.48611111111111116
TRAIN | Epoch Loss: 8.171371038250592e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.8356656432151794
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 4.0621983706452216e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.732540488243103
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 3.5023405741542645e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.746879776318868
VALID | Epoch Accuracy: 0.458333333

TRAIN | Epoch Loss: 3.826192408139052e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.6992082595825195
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 7.565532320898325e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 3.2832745711008706
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 4.6677532866721905e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.7286194960276287
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 3.796106336397186e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.9813749392827353
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 7.638477044175878e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 3.441253344217936
VALID | Epoch Accuracy: 0.4861111111111111
TRAIN | Epoch Loss: 3.0097507338463663e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.374672810236613
VALID | Epoch Accuracy: 0.45833333333

TRAIN | Epoch Loss: 5.360018570042614e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.9730517864227295
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 2.8668415065372494e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.701283613840739
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 3.778792597586289e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.769523228208224
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 5.0123249171260144e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.0883500973383584
VALID | Epoch Accuracy: 0.4583333333333334
TRAIN | Epoch Loss: 5.123586847080982e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 5.400651335716248
VALID | Epoch Accuracy: 0.4583333333333333
TRAIN | Epoch Loss: 0.00010382845019713776
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.7232557535171509
VALID | Epoch Accuracy: 0.458333333333

TRAIN | Epoch Loss: 3.389943295977511e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.6982989311218262
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 6.37230401220482e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 3.27400279045105
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 0.00012094775567155531
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.709413965543111
VALID | Epoch Accuracy: 0.45833333333333326
TRAIN | Epoch Loss: 4.840180970079798e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 2.8910348415374756
VALID | Epoch Accuracy: 0.47222222222222227
TRAIN | Epoch Loss: 5.2395322200027294e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.6939843098322551
VALID | Epoch Accuracy: 0.47222222222222215
TRAIN | Epoch Loss: 6.047175104738146e-05
TRAIN | Epoch Accuracy: 0.8928571428571429
VALID | Epoch Loss: 1.673378626505534
VALID | Epoch Accuracy: 0.4861111111111

In [ ]:
### LEFT OFF HERE.

# Your training loss goes down. Your validation loss stops improving quickly.  Would you call this overfitting? 

'''
- should you make sure you convert labels to longtenors? Are they Bytes now? 
- try starting with a lower learning rate?
- replace adam with SGD
- why is there no regularization in here? 
- this never gets a high accuracy. Seems random. Is the problem that it's just moving around the training space too much?
- what errors might have you made in defining your training loop?
- Can you add tqdm
- should you try SGD with OneCycle?  Is there a simpler scheduler? 
- Can you do a learning rate finder in pytorch?
- What if you quickly rewrote this code to use the new version of fast.ai? 
'''


### Perform Transfer Learning - Resnet18

* Download pre-trained model
* Freeze all but the last layer.
* Train for a little while
* Stop right around the overfitting line
* Reduce the learning rate
* Retrain all


Notes:

* still no drop out.
* can you do data augmentation

### Use Fast.ai

* Example of working with overhead data - https://github.com/fastai/fastai/blob/master/dev_nbs/course/lesson3-planet.ipynb
* Working with existing pytorch code - https://docs.fast.ai/migrating_pytorch

**Update** I moved this section to another file.

LEFT OFF HERE...IT LOOKS LIKE THERE'S MORE TEST DATA THAN TRAINING DATA. DID I MESS THIS UP?


In [ ]:
from fastai.vision.all import *
#from migrating_pytorch import *

In [ ]:
data = DataLoaders(train_dl, test_dl)
learn = Learner(data, Net(), loss_func=F.nll_loss, opt_func=Adam, metrics=accuracy, cbs=CudaCallback)

In [ ]:
learn.fit_one_cycle(epochs, lr)

### Testing

In [12]:
dataiter = iter(test_dl)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0])


In [21]:
for bi, b in enumerate(test_dl,0):
    labels, images = b
    print(labels)
    print(bi)
    break

tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0])
0


In [26]:
#messing with Subset
test_ds  = Subset(labeled_ds, test_idxs)
test_dl2 = DataLoader(test_ds, batch_size=bs)

dataiter = iter(test_dl2)
labels, images = dataiter.next()
print(images.shape)
print(labels)

torch.Size([24, 3, 32, 32])
tensor([0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1])
